In [2]:
import torch
from preprocessing import Preprocessor
from training import Trainer, load_partial_model, load_full_model
from metrics import Metrics
import torch.nn as nn
import torch.optim as optim
#from models.triple_arch import N_Parallel_Models
from models.vgg_TL import GoogleNet
#from models.autoencoders import Simple_AE, CNN_VAE
from configuration import Hyperparameters as HP
import json
import math
import tabulate

years = [str(y) for y in range(2006, 2015)]

classes_30_cf = ["Asterionellopsis", "bad", "Chaetoceros", "Ciliate_mix", "Corethron", "Cylindrotheca", "Dictyocha","dino30", "detritus", 
    "Chaetoceros_flagellate", "Dinobryon", "Ditylum", "Eucampia", "flagellate_sp3", "Guinardia_delicatula", "Guinardia_flaccida", "Guinardia_striata",
    "Heterocapsa_triquetra", "Laboea_strobila", "Leptocylindrus","pennate", "Phaeocystis", "Pleurosigma", "Prorocentrum", "Pseudonitzschia", 
    "Skeletonema", "Thalassionema", "Thalassiosira", "Thalassiosira_dirty", "Tintinnid"]


In [3]:
# original preprocessor
pp = Preprocessor(years, include_classes=classes_30_cf, strategy = HP.pp_strategy, train_eg_per_class=HP.number_of_images_per_class, maxN = HP.maxN, 
    minimum =  HP.minimum, transformations = HP.transformations)

pp.create_datasets(HP.data_splits)

trainLoader = pp.get_loaders('train', HP.batch_size)
validLoader = pp.get_loaders('validation', HP.batch_size)
testLoader = pp.get_loaders('test', HP.batch_size)

777655
29149


In [4]:
# confident images Preprocessor

path_to_statedict = "models/GoogleNet_5.3-13.31.pth"
model=GoogleNet(v=5.3)
if ".tar" in path_to_statedict:
    model = load_partial_model(model, path_to_statedict)
else:
    model = load_full_model(model, path_to_statedict)

trainer = Trainer(HP_version = 13.31, epochs = 0, loss_fn = HP.loss_function, optimizer = HP.optimizer)


    
test_pred, test_target, test_fnames, test_extras = trainer.test(model, testLoader, return_softmax=True, return_confs=True)
test_fnames, test_dropped_fnames = test_fnames
valid_pred, valid_target, valid_fnames, _ = trainer.test(model, validLoader)
valid_fnames, valid_dropped_fnames = valid_fnames
train_pred, train_target, train_fnames, _ = trainer.test(model, trainLoader)
train_fnames, train_dropped_fnames = train_fnames

cf_train_pp = Preprocessor(strategy="confident_images", conf_fnames=train_fnames)
cf_valid_pp = Preprocessor(strategy="confident_images", conf_fnames=valid_fnames)
cf_test_pp = Preprocessor(strategy="confident_images", conf_fnames=test_fnames)

/home/czm/saad/urfp-hab-plankton/models/vgg_TL.py:112: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  results = self.softmax(self.classifier(x))


17419
5805
5794


In [6]:
headers = ['classes', 'og_pp', 'conf_pp-train','conf_pp-valid','conf_pp-test']
data = []
for c in classes_30_cf:
    data.append([c, pp.label_to_onehotInd(c), cf_train_pp.label_to_onehotInd(c), cf_valid_pp.label_to_onehotInd(c), cf_test_pp.label_to_onehotInd(c)])

In [12]:
print(tabulate.tabulate(data,headers))

classes                   og_pp    conf_pp-train    conf_pp-valid    conf_pp-test
----------------------  -------  ---------------  ---------------  --------------
Asterionellopsis              0                0                0               0
bad                          25               25               25              25
Chaetoceros                   1                1                1               1
Ciliate_mix                   3                3                3               3
Corethron                     4                4                4               4
Cylindrotheca                 5                5                5               5
Dictyocha                     6                6                6               6
dino30                       27               27               27              27
detritus                     26               26               26              26
Chaetoceros_flagellate        2                2                2               2
Dinobryon       